# Backtest Parameter Grid (Local Jupyter)This notebook runs parameter grid explorations over the trading strategy using the existing backtest engine, **for local Jupyter**.Steps:1. Start Jupyter from the project root (where `src/` lives).2. Open this notebook from the `notebooks/` directory.3. Run the cells from top to bottom.

# Initialize

In [1]:
from pathlib import Path
import sys

# Assume this notebook lives in '<project_root>/notebooks'.
# Derive the project root as the parent directory and ensure it is on sys.path
# so that 'src' is importable without changing the process working directory.
project_root = Path().resolve().parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print("Project root:", project_root)
print("Has src?:", (project_root / "src").exists())

Project root: C:\Users\Anton\SRC\my\ml_lstm
Has src?: True


In [2]:
import itertools
import os

import numpy as np
import pandas as pd

from src.backtest import (
    _compute_atr_series,
    _compute_backtest_metrics,
    _make_naive_prediction_provider,
    _make_model_prediction_provider,
    _load_predictions_csv,
    _make_csv_prediction_provider,
)
from src.backtest_engine import BacktestConfig, run_backtest
from src.trading_strategy import StrategyConfig
from src.config import FREQUENCY, get_hourly_data_csv_path, get_predictions_csv_path

In [3]:
# Configuration for this notebook run
symbol = "nvda"
frequency = FREQUENCY  # Default from global config; override here if desired, e.g. "60min"
initial_equity = 10_000.0
prediction_mode = "csv"  # "naive", "model", or "csv"

csv_path = get_hourly_data_csv_path(frequency)
print("Using OHLC data from:", csv_path)
data = pd.read_csv(csv_path)

# Basic sanity check
required_cols = {"Open", "High", "Low", "Close"}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f"Data file {csv_path} is missing required columns: {missing}")

# Compute ATR(14) and a scalar ATR proxy
atr_series = _compute_atr_series(data, window=14)
atr_like = float(atr_series.dropna().mean()) if not atr_series.dropna().empty else 1.0
print(f"Mean ATR proxy: {atr_like:.4f}")

# Build prediction provider
if prediction_mode == "naive":
    provider = _make_naive_prediction_provider(offset_multiple=2.0, atr_like=atr_like)
elif prediction_mode == "model":
    provider = _make_model_prediction_provider(data, frequency=frequency)
elif prediction_mode == "csv":
    predictions_csv = get_predictions_csv_path(symbol, frequency)
    print("Using predictions CSV:", predictions_csv)
    preds_df = _load_predictions_csv(predictions_csv)
    provider = _make_csv_prediction_provider(preds_df, data)
else:
    raise ValueError(f"Unknown prediction_mode: {prediction_mode}")

Using OHLC data from: C:\Users\Anton\SRC\my\ml_lstm\data\processed\nvda_15min.csv
Mean ATR proxy: 0.3771
Using predictions CSV: C:\Users\Anton\SRC\my\ml_lstm\backtests\nvda_15min_predictions.csv


In [4]:
def run_one(
    strat_cfg: StrategyConfig,
    commission_per_unit_per_leg: float = 0.005,
    min_commission_per_order: float = 1.0,
):
    '''Run a single backtest with the given strategy and commission settings.'''
    bt_cfg = BacktestConfig(
        initial_equity=initial_equity,
        strategy_config=strat_cfg,
        model_error_sigma=atr_like,
        fixed_atr=atr_like,
        commission_per_unit_per_leg=commission_per_unit_per_leg,
        min_commission_per_order=min_commission_per_order,
    )

    result = run_backtest(
        data=data,
        prediction_provider=provider,
        cfg=bt_cfg,
        atr_series=atr_series,
        model_error_sigma_series=atr_series,
    )

    metrics = _compute_backtest_metrics(
        result,
        initial_equity=initial_equity,
        data=data,
    )

    row = {
        "final_equity": result.final_equity,
        "n_trades": len(result.trades),
        **metrics,
    }
    return result, row

# Grid A: risk_per_trade_pct × reward_risk_ratio

In [7]:
# Grid A: risk_per_trade_pct × reward_risk_ratio
risk_grid = [0.01, 0.02, 0.03]  # 0.0025, 0.005, 0.01, 0.03
rr_grid = [3.5, 4.0]  # [1.5, 2.0, 3.0]

rows = []
for risk_pct, rr in itertools.product(risk_grid, rr_grid):
    strat = StrategyConfig(
        risk_per_trade_pct=risk_pct,
        reward_risk_ratio=rr,
        k_sigma_err=0.5,
        k_atr_min_tp=3,
    )
    _, res_row = run_one(strat)
    res_row.update(
        {
            "grid": "risk_rr",
            "risk_per_trade_pct": risk_pct,
            "reward_risk_ratio": rr,
            "k_sigma_err": strat.k_sigma_err,
            "k_atr_min_tp": strat.k_atr_min_tp,
        }
    )
    rows.append(res_row)

df_risk_rr = pd.DataFrame(rows).sort_values("sharpe_ratio", ascending=False)
df_risk_rr.head(10)

,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor,grid,risk_per_trade_pct,reward_risk_ratio,k_sigma_err,k_atr_min_tp
10,42615.937603,84,3.261594,0.652404,-0.211080,1.543120,0.369048,1.727223,risk_rr,0.03,3.5,0.5,3
6,27306.858229,84,1.730686,0.416273,-0.144589,1.541677,0.369048,1.803601,risk_rr,0.02,3.5,0.5,3
2,16800.406302,84,0.680041,0.196912,-0.075133,1.531795,0.369048,1.887775,risk_rr,0.01,3.5,0.5,3
9,29635.352768,86,1.963535,0.456999,-0.226167,1.310336,0.372093,1.579706,risk_rr,0.03,3.0,0.5,3
5,21253.962522,86,1.125396,0.298500,-0.155198,1.307446,0.372093,1.627942,risk_rr,0.02,3.0,0.5,3
1,14742.734990,86,0.474273,0.143942,-0.080866,1.292047,0.372093,1.672658,risk_rr,0.01,3.0,0.5,3
11,34113.195803,100,2.411320,0.529790,-0.277736,1.221729,0.300000,1.497818,risk_rr,0.03,4.0,0.5,3
7,23782.367781,100,1.378237,0.350063,-0.191458,1.220410,0.300000,1.548448,risk_rr,0.02,4.0,0.5,3
8,23653.985931,79,1.365399,0.347534,-0.218245,1.210206,0.405063,1.484158,risk_rr,0.03,2.5,0.5,3
3,15748.465000,100,0.574847,0.170398,-0.100296,1.209053,0.300000,1.600995,risk_rr,0.01,4.0,0.5,3


# Grid B: model trust (k_sigma_err) × noise filter (k_atr_min_tp)

In [8]:
# Grid B: model trust (k_sigma_err) × noise filter (k_atr_min_tp)
k_sigma_grid = [0.25, 0.5, 0.75, 1.0]
k_atr_grid = [1.5, 2.0, 2.5, 3.0, 3.5]

rows = []
for k_sigma, k_atr_min_tp in itertools.product(k_sigma_grid, k_atr_grid):
    strat = StrategyConfig(
        risk_per_trade_pct=0.02,
        reward_risk_ratio=3.5,
        k_sigma_err=k_sigma,
        k_atr_min_tp=k_atr_min_tp,
    )
    _, res_row = run_one(strat)
    res_row.update(
        {
            "grid": "noise_filters",
            "risk_per_trade_pct": strat.risk_per_trade_pct,
            "reward_risk_ratio": strat.reward_risk_ratio,
            "k_sigma_err": k_sigma,
            "k_atr_min_tp": k_atr_min_tp,
        }
    )
    rows.append(res_row)

df_noise = pd.DataFrame(rows).sort_values("sharpe_ratio", ascending=False)
df_noise.head(10)

,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor,grid,risk_per_trade_pct,reward_risk_ratio,k_sigma_err,k_atr_min_tp
7,40483.777190,99,3.048378,0.623280,-0.161862,1.910401,0.393939,2.114007,noise_filters,0.02,3.5,0.50,2.5
2,37988.052895,115,2.798805,0.587887,-0.173406,1.761737,0.365217,1.907052,noise_filters,0.02,3.5,0.25,2.5
6,40183.099755,129,3.018310,0.619093,-0.161862,1.757574,0.356589,1.704803,noise_filters,0.02,3.5,0.50,2.0
16,36828.490264,112,2.682849,0.570924,-0.190078,1.741343,0.366071,1.858957,noise_filters,0.02,3.5,1.00,2.0
11,35754.413047,122,2.575441,0.554898,-0.173435,1.666348,0.352459,1.703386,noise_filters,0.02,3.5,0.75,2.0
3,29752.279762,97,1.975228,0.458988,-0.173406,1.577026,0.360825,1.828872,noise_filters,0.02,3.5,0.25,3.0
8,27306.858229,84,1.730686,0.416273,-0.144589,1.541677,0.369048,1.803601,noise_filters,0.02,3.5,0.50,3.0
12,29398.108500,106,1.939811,0.452947,-0.173435,1.521067,0.349057,1.690137,noise_filters,0.02,3.5,0.75,2.5
19,24625.875508,72,1.462588,0.366464,-0.173346,1.480319,0.375000,1.935685,noise_filters,0.02,3.5,1.00,3.5
9,22534.578455,72,1.253458,0.325089,-0.158212,1.362316,0.361111,1.731953,noise_filters,0.02,3.5,0.50,3.5


In [9]:
df_noise.head(30)

,final_equity,n_trades,total_return,cagr,max_drawdown,sharpe_ratio,win_rate,profit_factor,grid,risk_per_trade_pct,reward_risk_ratio,k_sigma_err,k_atr_min_tp
7,40483.777190,99,3.048378,0.623280,-0.161862,1.910401,0.393939,2.114007,noise_filters,0.02,3.5,0.50,2.5
2,37988.052895,115,2.798805,0.587887,-0.173406,1.761737,0.365217,1.907052,noise_filters,0.02,3.5,0.25,2.5
6,40183.099755,129,3.018310,0.619093,-0.161862,1.757574,0.356589,1.704803,noise_filters,0.02,3.5,0.50,2.0
16,36828.490264,112,2.682849,0.570924,-0.190078,1.741343,0.366071,1.858957,noise_filters,0.02,3.5,1.00,2.0
11,35754.413047,122,2.575441,0.554898,-0.173435,1.666348,0.352459,1.703386,noise_filters,0.02,3.5,0.75,2.0
3,29752.279762,97,1.975228,0.458988,-0.173406,1.577026,0.360825,1.828872,noise_filters,0.02,3.5,0.25,3.0
8,27306.858229,84,1.730686,0.416273,-0.144589,1.541677,0.369048,1.803601,noise_filters,0.02,3.5,0.50,3.0
12,29398.108500,106,1.939811,0.452947,-0.173435,1.521067,0.349057,1.690137,noise_filters,0.02,3.5,0.75,2.5
19,24625.875508,72,1.462588,0.366464,-0.173346,1.480319,0.375000,1.935685,noise_filters,0.02,3.5,1.00,3.5
9,22534.578455,72,1.253458,0.325089,-0.158212,1.362316,0.361111,1.731953,noise_filters,0.02,3.5,0.50,3.5
